In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import requests

In [0]:
current_version = spark.sql("DESCRIBE HISTORY db_workspace.my_schema.revenues_per_day").selectExpr("max(version)").collect()[0][0]

schema = StructType([
    StructField("actors", StringType(), True),
    StructField("imdbRating", StringType(), True),
    StructField("imdbVotes", StringType(), True),
])

@F.udf(schema)
def from_url(title):
    api_url = f"https://www.omdbapi.com/?t={title}&apikey=b2715e56"
    response = requests.get(api_url)
    movie_data = response.json()
    data1 = movie_data.get('Actors', None)
    data2 = movie_data.get('imdbRating', None)
    data3 = movie_data.get('imdbVotes', None)
    return (data1, data2, data3)

In [0]:
df = (
    spark.read
        .option('readChangeData', 'true')
        .option("startingVersion", current_version)
        .table('db_workspace.my_schema.revenues_per_day')
        .drop('id')
        .withColumn('download_date', F.current_date())
        .withColumn('from_url', from_url('title'))
)

df.write.mode('overwrite').saveAsTable('db_workspace.my_schema.temp')